In [1]:
import os
import math
from dotenv import load_dotenv

# Explicitly providing path to '.env'
from pathlib import Path  # Python 3.6+ only
# Load .env variables
_ = load_dotenv(dotenv_path=f"{Path().resolve().parents[1]}/standalone/.env")

import import_ipynb
from utils import to_timestamp, df_data_sensors, to_be_checked, is_corr_ok, df_to_csv, df_from_csv

importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from aqi_epa_pm25.ipynb
PM2.5: 35.9, AQI: 102
PM2.5: 35.9, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 35.5, Max: 55.4
AQI: 102, Measure Level: MeasureLevels.UNHEALTHY_FOR_SENSITIVE_GROUPS, Range Values: Min: 101, Max: 150


## Humidity Raw Data

In [2]:
# Load Tangaras
df_tangaras = df_from_csv("tangaras.csv", dtindex=False)

print(f"Total Tangara Sensors: {len(df_tangaras)}")

df_tangaras.head()

Total Tangara Sensors: 14


,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.384476,-76.516342
1,TANGARA_260A,d29edyj,D29ESP32DE1260A,3.4613800048828125 -76.51222229003906,3.461380,-76.512222
2,TANGARA_4B1A,d29esj8,D29ESP32DE94B1A,3.4586334228515625 -76.46415710449219,3.458633,-76.464157
3,TANGARA_14D6,d29eg4k,D29ESP32DED14D6,3.4847259521484375 -76.50260925292969,3.484726,-76.502609
4,TANGARA_1EE6,d29edh3,D29ESP32DED1EE6,3.4517669677734375 -76.55067443847656,3.451767,-76.550674


In [3]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
START_ISO8601_DATETIME = os.getenv("START_ISO8601_DATETIME", None)
start_timestamp = to_timestamp(START_ISO8601_DATETIME)
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
END_ISO8601_DATETIME = os.getenv("END_ISO8601_DATETIME", None)
end_timestamp = to_timestamp(os.getenv("END_ISO8601_DATETIME", None))

# GROUP BY TIME
GROUP_BY_TIME = os.getenv("GROUP_BY_TIME", None)

print(f'Since: {START_ISO8601_DATETIME} -> {start_timestamp}, Until: {END_ISO8601_DATETIME} -> {end_timestamp}')
print(f"Group by Time: {GROUP_BY_TIME}")

Since: 2023-10-05T22:58:24-05:00 -> 1696564704000, Until: 2023-10-06T22:58:24-05:00 -> 1696651104000
Group by Time: 30s


In [4]:
# Humidity Data Frame Sensors
df_hum_raw = df_data_sensors(df_tangaras, start_timestamp, end_timestamp, 'hum', GROUP_BY_TIME)
df_hum_raw.head()
#df_hum_raw.shape

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
DATETIME,,,,,,,,,,,,,,
2023-10-05 22:58:30-05:00,0.0,57.98,62.07,NaN,NaN,67.28,62.92,NaN,58.30,64.59,99.9,98.8,51.42,75.82
2023-10-05 22:59:00-05:00,0.0,58.05,62.14,NaN,NaN,67.76,63.32,NaN,58.50,64.42,99.9,98.7,51.50,74.95
2023-10-05 22:59:30-05:00,0.0,57.99,62.20,NaN,NaN,66.68,62.83,NaN,58.84,64.52,99.9,98.6,51.58,75.42
2023-10-05 23:00:00-05:00,0.0,58.19,62.26,NaN,NaN,67.47,62.99,NaN,58.85,64.30,99.9,98.4,51.59,76.60
2023-10-05 23:00:30-05:00,0.0,57.21,62.32,NaN,NaN,67.83,63.00,NaN,58.86,64.67,99.9,98.3,51.74,76.90


## Descriptive Statistics

In [5]:
# Describe Data
df_hum_raw.describe()

,TANGARA_2BBA,TANGARA_260A,TANGARA_4B1A,TANGARA_14D6,TANGARA_1EE6,TANGARA_2B42,TANGARA_2E9A,TANGARA_2FF6,TANGARA_307A,TANGARA_1712,TANGARA_48C6,TANGARA_532E,TANGARA_F1AE,TANGARA_06BE
count,2708.0,2619.000000,2666.000000,1303.000000,2552.000000,2333.000000,2579.000000,2535.000000,2647.000000,2666.000000,2602.000000,2697.000000,2693.000000,2682.000000
mean,0.0,56.474475,61.534932,48.275986,56.406230,62.466931,57.740415,67.176978,53.409860,61.974692,99.566410,92.254060,53.799822,61.611581
std,0.0,8.490096,10.079412,6.152645,4.781258,8.930612,6.258782,12.288505,11.267487,9.320114,0.815928,7.786387,8.057845,11.359806
min,0.0,42.570000,42.690000,38.620000,48.920000,41.110000,44.500000,45.980000,32.050000,44.200000,96.100000,74.100000,40.340000,43.090000
25%,0.0,50.570000,54.140000,41.220000,51.790000,57.160000,53.045000,56.040000,41.105000,54.182500,99.900000,90.000000,47.540000,50.025000
50%,0.0,55.800000,62.525000,49.770000,57.415000,62.260000,59.160000,66.800000,57.560000,65.070000,99.900000,93.600000,53.710000,60.980000
75%,0.0,65.870000,72.447500,52.515000,61.170000,71.960000,63.110000,80.080000,62.585000,69.825000,99.900000,98.200000,62.490000,72.920000
max,0.0,69.830000,76.610000,59.990000,61.860000,74.910000,65.240000,82.860000,69.030000,74.290000,99.900000,99.900000,66.820000,77.490000


## Missing Data

In [6]:
# Missing Data Threshold
threshold = 90
print(f'Threshold: {threshold}%')

# For each Tangara sensor
for id_tangara_sensor in df_hum_raw.columns:
    df_hum_sensor = df_hum_raw[[id_tangara_sensor]]
    # To be checked
    is_ok, data_percent, missing_data_percent = to_be_checked(df_hum_sensor, threshold)
    if not is_ok:
        print(f"Tangara Sensor: {id_tangara_sensor}, Data: {data_percent}%, Missing: {missing_data_percent}%, To be checked")

Threshold: 90%
Tangara Sensor: TANGARA_14D6, Data: 45%, Missing: 55%, To be checked
Tangara Sensor: TANGARA_1EE6, Data: 89%, Missing: 11%, To be checked
Tangara Sensor: TANGARA_2B42, Data: 81%, Missing: 19%, To be checked
Tangara Sensor: TANGARA_2FF6, Data: 88%, Missing: 12%, To be checked


## Data Correlation

In [7]:
# Data Correlation Threshold
threshold = 0.9
print(f'Threshold: {threshold}%')

ID_REFE_TANGARA_SENSOR='TANGARA_06BE'
ID_TARG_TANGARA_SENSOR='TANGARA_2FF6'
# Reference Tangara Sensor
df_reference_sensor = df_hum_raw[ID_REFE_TANGARA_SENSOR]
# Target Tangara Sensor
df_target_sensor = df_hum_raw[ID_TARG_TANGARA_SENSOR]

# To be checked
if not math.isnan(df_reference_sensor.std()):
    is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
    if not is_ok:
        print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {ID_TARG_TANGARA_SENSOR}, Correlation: {corr}, To be checked")

# For each Tangara sensor
# for id_tangara_sensor in df_hum_raw.columns:
#     # Target Tangara Sensor
#     df_target_sensor = df_hum_raw[id_tangara_sensor]
#     # To be checked
#     is_ok, corr = is_corr_ok(df_reference_sensor, df_target_sensor, threshold)
#     if not is_ok:
#         print(f"Reference Tangara Sensor: {ID_REFE_TANGARA_SENSOR}, Target Tangara Sensor: {id_tangara_sensor}, Correlation: {corr}, To be checked")


Threshold: 0.9%
Reference Tangara Sensor: TANGARA_06BE, Target Tangara Sensor: TANGARA_2FF6, Correlation: 0.0, To be checked


In [8]:
# Save Humidity Data Frame Sensors into CSV file
df_to_csv(df_hum_raw, "hum_raw.csv")